In [29]:
import shutil
import os

# Directory to delete
EXTRACT_DIR = 'csvs'

# Check if the directory exists and delete it
if os.path.isdir(EXTRACT_DIR):
    shutil.rmtree(EXTRACT_DIR)
    print(f"Directory '{EXTRACT_DIR}' has been deleted.")
else:
    print(f"Directory '{EXTRACT_DIR}' does not exist.")

Directory 'csvs' has been deleted.


In [1]:
import zipfile
import shutil
import os
import pandas as pd
print("CPU cores: ", os.cpu_count())

CPU cores:  144


In [2]:
def extract_zip(zip_path):

    if not zipfile.is_zipfile(zip_path):
        raise ValueError(f"The file at {zip_path} is not a valid ZIP archive.")
    
    # Determine the output directory name from the zip file name
    base_dir = os.path.dirname(zip_path)
    zip_filename = os.path.basename(zip_path)
    folder_name = os.path.splitext(zip_filename)[0]
    extract_to = os.path.join(base_dir, folder_name)

    # Create the directory if it doesn't exist
    if not os.path.exists(extract_to):
        os.makedirs(extract_to)

    # Extract the ZIP file
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(path=extract_to)

    return extract_to


if __name__ == "__main__":
    zip_file_path = '0.zip' 
    extracted_dir = extract_zip(zip_file_path)
    print(f"Extracted to: {extracted_dir}")

Extracted to: 0


In [3]:
leak_directory = "0"

# Path to the __MACOSX folder
macosx_folder = os.path.join(leak_directory, "__MACOSX")

# Check if __MACOSX exists and remove it - creates issues when analyzing the data, and its not needed, made automatically by MacOS
if os.path.exists(macosx_folder) and os.path.isdir(macosx_folder):
    shutil.rmtree(macosx_folder)
    print(f"Deleted - {macosx_folder}")
else:
    print(f"Folder not found - {macosx_folder}")

# Organize files by extension into subfolders
for root, dirs, files in os.walk(leak_directory):
    for file in files:
        # Skip hidden files and __MACOSX if any reappear
        if file.startswith('.') or '__MACOSX' in root:
            continue

        # Get the file extension (in lowercase, without the dot)
        file_extension = os.path.splitext(file)[1].lower().lstrip('.')
        if not file_extension:
            file_extension = "no_extension"

        # Define the new subfolder path
        subfolder_path = os.path.join(leak_directory, file_extension)

        # Create the subfolder if it doesn't exist
        os.makedirs(subfolder_path, exist_ok=True)

        # Define source and destination paths
        source_path = os.path.join(root, file)
        destination_path = os.path.join(subfolder_path, file)

        # Move the file if source and destination are not the same
        if os.path.abspath(source_path) != os.path.abspath(destination_path):
            shutil.move(source_path, destination_path)

# Remove any empty folders within the parent directory
for dirpath, dirnames, filenames in os.walk(leak_directory, topdown=False):
    if not dirnames and not filenames:
        try:
            os.rmdir(dirpath)
            print(f"Removed empty folder: {dirpath}")
        except OSError:
            pass  

Deleted - 0/__MACOSX


In [4]:
# Delete Duplicate of 0 which is empty

# Directory to delete
EXTRACT_DIR = '0/0'

# Check if the directory exists and delete it
if os.path.isdir(EXTRACT_DIR):
    shutil.rmtree(EXTRACT_DIR)
    print(f"Directory '{EXTRACT_DIR}'deleted")
else:
    print(f"Directory '{EXTRACT_DIR}' does not exist")

Directory '0/0'deleted


# Dataframe with all file location

In [5]:
def get_folder_file_dataframe(root_dir):
    folders = []
    for file_name in os.listdir(root_dir):
        full_path = os.path.join(root_dir, file_name)
        if os.path.isdir(full_path):
            folders.append(file_name)

    series_list = []
    for folder in folders:
        folder_path = os.path.join(root_dir, folder)
        files = [
            os.path.join(folder_path, file)
            for file in os.listdir(folder_path)
            if os.path.isfile(os.path.join(folder_path, file)) and not file.startswith('.')
        ]
        s = pd.Series(files, name=folder)
        series_list.append(s)

    df = pd.concat(series_list, axis=1)
    return df

In [6]:
df = get_folder_file_dataframe(leak_directory)
df.head(30)       

,md,png,log,txt
0,0/md/dbc9c90e-a3e6-4d71-bb93-5fb8394095ac.md,0/png/64bba692-d430-440c-9f1e-2575f45770af_6.png,0/log/77010155050.log,0/txt/IDNET.txt
1,0/md/28.md,0/png/12756724-394c-4576-b373-7c53f1abbd94_0.png,0/log/77753527617.log,0/txt/IDTV.txt
2,0/md/5.md,0/png/f179eb06-0c53-44df-a13f-570be23355bb_1.png,0/log/tele2-lbs.log,0/txt/beeline-77774042222.txt
3,0/md/38.md,0/png/5a6b122c-39c1-4581-8c1f-2d6f36a9f8a0_24.png,0/log/tele2-cdr.log,0/txt/beeline-77051056626.txt
4,0/md/9d7bc879-3250-4013-ac04-5ff9bd6dff40.md,0/png/5a6b122c-39c1-4581-8c1f-2d6f36a9f8a0_30.png,0/log/tele2-crm.log,0/txt/beeline-crm.txt
5,0/md/18.md,0/png/5a6b122c-39c1-4581-8c1f-2d6f36a9f8a0_18.png,0/log/77783030133.log,0/txt/UBSCRIBER.txt
6,0/md/9fd06037-11f1-4ad5-9a7d-cbfb3fa4193b.md,0/png/0-adaf869e-920a-4a17-91bd-e2ef3125c10e.png,NaN,0/txt/beeline-cdr.txt
7,0/md/3348953d-66e9-4cac-8675-65bb5f2ef929.md,0/png/5387a301-0af8-4e24-a197-20189f87b9ef_8.png,NaN,0/txt/CRM.txt
8,0/md/1.md,0/png/0-32eb7662-f212-4811-a7c1-1cfeb121cd99.png,NaN,0/txt/LAC.txt
9,0/md/19.md,0/png/912204cb-8ab7-48b8-9abf-d803f3804d08_11.png,NaN,0/txt/beeline-lbs.txt


# Use LLM to classify the source files

In [7]:
import os
import concurrent.futures
import pandas as pd
from langchain.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from tqdm import tqdm

In [10]:
llm = Ollama(model="llama3.1:8b")

prompt_template = PromptTemplate(
    input_variables=["content"],
    template="""
You are analyzing the content of a file.

File content:
\"\"\"
{content}
\"\"\"

1. Classify the content into one of the following categories ONLY: chats, images, other.
2. State your confidence in the classification as one of: high, medium, or low.
3. I will have to make a csv, please give me a list of headers based on the content. E.g. "[<header_name>, <heaer_name2>, etc.]
Respond in the following format:
Category: <chats|images|other>
Confidence: <high|medium|low>
"""
)

chain = LLMChain(llm=llm, prompt=prompt_template)

# Preprocess the files to the first 20 lines to shorten analysis times
def preprocess_first_20_lines(file_path):
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            lines = [line.strip() for _, line in zip(range(20), f)]
            content = " ".join(lines)
        return file_path, content
    except Exception:
        return file_path, ""

# Use only the first column of the DataFrame
first_column = df.columns[0]
file_paths = df[first_column].dropna().unique().tolist()

# Preprocess the file for faster classification
with concurrent.futures.ThreadPoolExecutor() as executor:
    file_data = list(executor.map(preprocess_first_20_lines, file_paths))

# Classify and collect by category
valid_categories = {"chats", "images", "other"}
categorized_files = {cat: [] for cat in valid_categories}

for file_path, content in tqdm(file_data, desc="Classifying files"):
    if not content:
        continue

    try:
        response = chain.run(content=content).strip().lower()
        lines = response.splitlines()

        category = ""
        for line in lines:
            if line.startswith("category:"):
                category = line.replace("category:", "").strip()
                break
        if category not in valid_categories:
            category = "other"

        categorized_files[category].append(file_path)

    except Exception as e:
        print(f"Error processing {file_path}: {e}")

# Convert to DataFrame (columns = categories)
# Determine the maximum list length among all categories
max_len = 0
for file_list in categorized_files.values():
    if len(file_list) > max_len:
        max_len = len(file_list)

# Pad each list with None to match max length
padded = {}
for category, file_list in categorized_files.items():
    padding_needed = max_len - len(file_list)
    padded[category] = file_list + [None] * padding_needed
result_df = pd.DataFrame(padded)
result_df.to_csv("classified_by_category.csv", index=False)
result_df.head()

Classifying files: 100%|██████████| 70/70 [02:51<00:00,  2.45s/it]


,images,other,chats
0,0/md/dbc9c90e-a3e6-4d71-bb93-5fb8394095ac.md,0/md/9d7bc879-3250-4013-ac04-5ff9bd6dff40.md,0/md/28.md
1,0/md/9fd06037-11f1-4ad5-9a7d-cbfb3fa4193b.md,None,0/md/5.md
2,0/md/3348953d-66e9-4cac-8675-65bb5f2ef929.md,None,0/md/38.md
3,0/md/07f179c5-5705-4dbd-94a7-66eed1e066b0.md,None,0/md/18.md
4,0/md/01cdc26f-e773-4ad7-8808-d04abf16aae7.md,None,0/md/1.md


In [11]:
import os
import pandas as pd
import re
from collections import defaultdict

# Script to check for cross refernced files

In [12]:
import pandas as pd
import os
import re
from collections import defaultdict

In [13]:
# Extract all unique file paths from the DataFrame 
file_paths = df.stack().dropna().unique().tolist()

# Read file content
def read_file(path):
    try:
        with open(path, 'r', encoding='utf-8') as f:
            return f.read()
    except:
        return "Failed to open file"

# Read file contents into a dictionary 
file_contents = {path: read_file(path) for path in file_paths}

# Build a reference map (which files mention which) 
reference_map = defaultdict(list)

for source_path, content in file_contents.items():
    if content is None:
        continue  # skip if content is None

    for target_path in file_paths:
        if target_path == source_path:
            continue  # skip comparing file to itself

        target_filename = os.path.basename(target_path)
        pattern = re.escape(target_filename)

        if re.search(rf'\b{pattern}\b', content):
            reference_map[source_path].append(target_path)

# Convert reference map to a DataFrame
ref_df = pd.DataFrame([
    {"source_file": src, "mentions": tgt}
    for src, tgts in reference_map.items()
    for tgt in tgts
])

# Group mentions into lists per source_file
ref_summary = ref_df.groupby("source_file")["mentions"].apply(list).reset_index()

# Some source_file names are also in mentions, therefore they aren't a source file anymore
source_files = set(ref_summary["source_file"])
mentioned_files = set(file for mention_list in ref_summary["mentions"] for file in mention_list)
common_files = source_files.intersection(mentioned_files)

# Filter out common files from the DataFrame
filtered_ref_summary = ref_summary[~ref_summary["source_file"].isin(common_files)].reset_index(drop=True)

filtered_ref_summary.to_csv("file_reference_map.csv", index=False)
print(filtered_ref_summary.head(10))
print("Unique file paths in original DataFrame:", df.stack().nunique())

                                    source_file  \
0  0/md/01cdc26f-e773-4ad7-8808-d04abf16aae7.md   
1                                    0/md/10.md   
2                                    0/md/13.md   
3                                    0/md/15.md   
4  0/md/178e3898-903d-47cf-bfbe-061e7dc18895.md   
5                                    0/md/19.md   
6                                     0/md/2.md   
7                                    0/md/20.md   
8                                    0/md/21.md   
9                                    0/md/22.md   

                                            mentions  
0  [0/png/01cdc26f-e773-4ad7-8808-d04abf16aae7_1_...  
1  [0/png/0-6bcc0131-e4ad-421e-bb1f-d8ebe5eeec7b....  
2  [0/png/0-adaf869e-920a-4a17-91bd-e2ef3125c10e....  
3  [0/png/0-b8cea3b1-4dde-4438-9b1a-6faf690bbad0....  
4  [0/png/178e3898-903d-47cf-bfbe-061e7dc18895_8....  
5  [0/png/6848748d-2881-4c26-b153-fcd5373d2f1c.pn...  
6  [0/md/07f179c5-5705-4dbd-94a7-66eed1e066b0.md,... 

# Check for more linkages in the files + add the missing source files from the chats

In [ ]:
import pandas as pd
import os
import re
from ast import literal_eval
import ast

In [16]:
def is_readable_text_file(path):
    return any(path.endswith(ext) for ext in ['txt', 'md', 'rst', 'tex', 'nfo', 'readme', 'rtf', 'doc', 'docx',
    'cfg', 'conf', 'config', 'ini', 'json', 'yaml', 'yml', 'toml',
    'log', 'lst', 'cnf', 'properties', 'prefs',
    'csv', 'tsv', 'dat', 'db', 'dbf', 'sql', 'xml',
    'ssv', 'psv', 'jsonl', 'parquet', 'orc',
    'html', 'htm', 'xhtml', 'xht', 'css', 'js',
    'jsx', 'ts', 'tsx', 'vue', 'erb', 'ejs', 'jsp',
    'liquid', 'handlebars', 'hbs', 'mustache',
    'py', 'pyw', 'ipynb', 'java', 'c', 'cpp', 'h', 'hpp', 'cs',
    'sh', 'bash', 'zsh', 'ksh', 'bat', 'cmd', 'ps1',
    'r', 'jl', 'pl', 'pm', 'rb', 'go', 'lua', 'php',
    'swift', 'scala', 'dart', 'asm', 'groovy',
    'rmd', 'sage', 'nb',
    'env', 'gradle', 'makefile', 'mak', 'mk',
    'dockerfile', 'gitignore', 'gitattributes', 'gitmodules',
    'cmake', 'make', 'ninja', 'build',
    'manifest', 'manifest.json', 'vtt', 'srt', 'resx', 'strings',
    'lang', 'po', 'mo', 'pot', 'msg', 'textbundle',
    'rego', 'tf', 'tfvars', 'cue', 'bzl', 'bazel', 'nix', 'dhall',
    'adoc', 'asciidoc', 'creole', 'mediawiki', 'wiki', 'org',
    'eml', 'msg', 'mbox', 'mail', 'ics', 'vcf'])

mentions2_list = []

# Iterate over each row of the summary
for _, row in filtered_ref_summary.iterrows():
    mention_dict = {}

    # For each mentioned file read file contents and create a dictionary key: location - values: file contents
    for mentioned_file in row["mentions"]:
        if not is_readable_text_file(mentioned_file):
            continue  # Skip unreadable formats

        # Read file content
        content = read_file(mentioned_file)
        if content in [None, "Failed to open file"]:
            continue

        # Search for any file-type references using regex
        # Match anything with a name and known file extension
        found_files = re.findall(r'[\w\-/]*\.\w+', content)

        # Filter duplicates and add to the mention_dict
        if found_files:
            mention_dict[mentioned_file] = list(set(found_files))

    mentions2_list.append(mention_dict)

# Add this as a new column
filtered_ref_summary["mentions2"] = mentions2_list

# Get the list of chat files from result_df - LLM clasification
chat_files = result_df['chats'].dropna().unique()

# Get the list of source files already in filtered_ref_summary
source_files = filtered_ref_summary['source_file'].dropna().unique()

# Find missing chat files and collect chat files that are not found in source files
existing_source_files = set(source_files)
missing_chat_files = []
for chat_file in chat_files:
    if chat_file not in existing_source_files:
        missing_chat_files.append(chat_file)

# Create a new DataFrame with empty mentions and linkage columns
new_rows = pd.DataFrame({
    'source_file': missing_chat_files,
})

# Append the new rows to ref_summary
filtered_ref_summary = pd.concat([filtered_ref_summary, new_rows], ignore_index=True)
        
# Save the updated DataFrame to CSV
filtered_ref_summary.to_csv("file_reference_map_extended.csv", index=False)
print(filtered_ref_summary.columns)

Index(['source_file', 'mentions', 'mentions2'], dtype='object')


# Structuring the Data into CSV - Step 1 all MD files

In [17]:
import os
import pandas as pd
from pathlib import Path
from bs4 import BeautifulSoup

In [18]:
# Create a folder and dumb all csvs
output_dir = "csvs"
os.makedirs(output_dir, exist_ok=True)

# Get all MD files from the DataFrame
all_file_paths = pd.unique(result_df.values.ravel('K'))
md_files = []
for file_path in all_file_paths:
    if isinstance(file_path, str) and file_path.endswith(".md") and os.path.exists(file_path):
        md_files.append(file_path)
        
# Use the LLM's clasification of MD files
chat_files = set(result_df['chats'].dropna().astype(str))

# Extract visible text + ANY file reference 
def extract_text_and_files(td):
    text = td.get_text(strip=True)
    links = []
    for tag in td.find_all(['a', 'img']):
        link = tag.get('href') or tag.get('src')
        if link:
            fname = os.path.basename(link.strip("'\""))
            links.append(fname)
    return text + (" " + " ".join(links) if links else "")

# This method extracts chat table from HTML
def process_chat_md(file_path):
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            content = f.read()
        soup = BeautifulSoup(content, "lxml")
        rows = soup.find_all("tr")[1:]
        data = []
        for row in rows:
            cols = row.find_all("td")
            if len(cols) == 4:
                data.append({
                    "Time": cols[0].get_text(strip=True),
                    "From": cols[1].get_text(strip=True),
                    "To": cols[2].get_text(strip=True),
                    "Message": extract_text_and_files(cols[3])
                })
        return pd.DataFrame(data) if data else None
    except Exception as e:
        print(f"Error processing chat file {file_path}: {e}")
        return None

# Extract the contents of the other file types that the LLM classified
def process_reference_md(file_path):
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            content = f.read()
        soup = BeautifulSoup(content, "lxml")
        matches = []
        for tag in soup.find_all(['a', 'img']):
            link = tag.get('href') or tag.get('src')
            if link:
                fname = os.path.basename(link.strip("'\""))
                if not fname.startswith("0-"):
                    matches.append(fname)
        matches = list(set(matches))
        return pd.DataFrame([{"File_Name": f} for f in matches]) if matches else None
    except Exception as e:
        print(f"Error processing reference file {file_path}: {e}")
        return None

# Loop through all MD filesand call the above methods for the afferent file type
for file_path in md_files:
    csv_name = os.path.basename(file_path).replace(".md", ".csv")
    csv_path = os.path.join(output_dir, csv_name)

    if not os.path.exists(csv_path):
        if file_path in chat_files:
            df = process_chat_md(file_path)
        else:
            df = process_reference_md(file_path)

        if df is not None:
            df.to_csv(csv_path, index=False)
        else:
            print(f"Couldn't save to CSV: {file_path}")

In [19]:
import os
import ast
from pathlib import Path

In [24]:
csv_dir = "csvs"

# Replace MD with CSV if the corresponding file exists
def replace_md_with_csv(item):
    if isinstance(item, str) and item.endswith(".md"):
        base_name = Path(item).stem  # .stem extracts the filename with no extension
        # CSV search
        csv_path = os.path.join(csv_dir, f"{base_name}.csv")
        if os.path.exists(csv_path):
            return csv_path
        else:
            print(f"CSV {item} not found")
    return item

# Recursively handle strings, lists, and dictionaries
def process_column_cell(cell):
    if isinstance(cell, str):
        return replace_md_with_csv(cell)
    
    elif isinstance(cell, list):
        new_list = []
        for item in cell:
            new_item = process_column_cell(item)
            new_list.append(new_item)
        return new_list

    elif isinstance(cell, dict):
        new_dict = {}
        for key, value in cell.items():
            new_key = replace_md_with_csv(key)
            new_value = process_column_cell(value)
            new_dict[new_key] = new_value
        return new_dict

    else:
        return cell
    
for col in filtered_ref_summary.columns:
    filtered_ref_summary[col] = filtered_ref_summary[col].apply(process_column_cell)

filtered_ref_summary.to_csv("filtered_ref_summary_csv_replaced.csv", index=False)
filtered_ref_summary.head()        

,source_file,mentions,mentions2
0,csvs/01cdc26f-e773-4ad7-8808-d04abf16aae7.csv,[0/png/01cdc26f-e773-4ad7-8808-d04abf16aae7_1_...,{}
4,csvs/178e3898-903d-47cf-bfbe-061e7dc18895.csv,[0/png/178e3898-903d-47cf-bfbe-061e7dc18895_8....,{}
10,csvs/23.csv,[csvs/547aba02-6757-49c1-acb5-6df217cebfc7.csv],{'csvs/547aba02-6757-49c1-acb5-6df217cebfc7.cs...
12,csvs/26.csv,[csvs/54990932-71af-48dd-9a7a-2617b1407c54.csv],{'csvs/54990932-71af-48dd-9a7a-2617b1407c54.cs...
13,csvs/2db27de1-d5c5-4f89-8572-da697a6329e4.csv,[0/png/2db27de1-d5c5-4f89-8572-da697a6329e4_5_...,{}


# Structuring the Data into CSV - Step 2 OCR on image files

In [19]:
import os

In [20]:
png_paths = []


# Gather every png path from the df
def gather_pngs(dataframe_name):
    for column in dataframe_name.columns:

        # Loop through each cell in that column
        for cell in dataframe_name[column]:

            # Case 1: The cell is a simple string
            if isinstance(cell, str):
                if cell.endswith(".png"):
                    png_paths.append(cell)

            # List
            elif isinstance(cell, list):
                for item in cell:
                    if isinstance(item, str) and item.endswith(".png"):
                        png_paths.append(item)

            # Dictionary
            elif isinstance(cell, dict):
                for key, value in cell.items():
                    # Check key
                    if isinstance(key, str) and key.endswith(".png"):
                        png_paths.append(key)

                    # Check value
                    if isinstance(value, str) and value.endswith(".png"):
                        png_paths.append(value)

                    # If the value is a list of paths
                    elif isinstance(value, list):
                        for sub_item in value:
                            if isinstance(sub_item, str) and sub_item.endswith(".png"):
                                png_paths.append(sub_item)
                                
gather_pngs(filtered_ref_summary)
print(len(png_paths))

488


# PaddleOCR

In [27]:
from paddleocr import PaddleOCR
import os

# Use the Chinese model
ocr_model = PaddleOCR(lang='ch')

def paddle_ocr(image_path):
    if not os.path.exists(image_path):
        print(f"File not found - {image_path}")
        return ""

    result = ocr_model.ocr(image_path)

    # For document structure output
    if isinstance(result, list) and isinstance(result[0], dict) and 'rec_texts' in result[0]:
        return "\n".join(result[0]['rec_texts'])

    # For basic detection output
    extracted_text = []
    for line in result[0]:
        text = line[1][0]  # text, confidence
        extracted_text.append(text)

    return "\n".join(extracted_text)

Creating model: ('PP-LCNet_x1_0_doc_ori', None)
Using official model (PP-LCNet_x1_0_doc_ori), the model files will be automatically downloaded and saved in /home/jovyan/.paddlex/official_models.
E0526 11:49:37.156906  2204 analysis_config.cc:169] Please use PaddlePaddle with GPU version.
Creating model: ('UVDoc', None)
Using official model (UVDoc), the model files will be automatically downloaded and saved in /home/jovyan/.paddlex/official_models.
E0526 11:49:37.669418  2204 analysis_config.cc:169] Please use PaddlePaddle with GPU version.
Creating model: ('PP-LCNet_x0_25_textline_ori', None)
Using official model (PP-LCNet_x0_25_textline_ori), the model files will be automatically downloaded and saved in /home/jovyan/.paddlex/official_models.
E0526 11:49:40.892796  2204 analysis_config.cc:169] Please use PaddlePaddle with GPU version.
Creating model: ('PP-OCRv5_mobile_det', None)
Using official model (PP-OCRv5_mobile_det), the model files will be automatically downloaded and saved in /

# EasyOCR

In [21]:
import easyocr
import os

In [22]:
# Use the Chinese and English model
ocr_model = easyocr.Reader(['ch_sim'], gpu = True)  # Load once

def easy_ocr(image_path):
    if not os.path.exists(image_path):
        print(f"File not found - {image_path}")
        return ""

    results = ocr_model.readtext(image_path, detail=0)  # Only return text, not boxes/confidences

    return "\n".join(results)

In [23]:
import shutil
import os

# Directory to delete
EXTRACT_DIR = 'ocr'

# Check if the directory exists and delete it
if os.path.isdir(EXTRACT_DIR):
    shutil.rmtree(EXTRACT_DIR)
    print(f"Directory '{EXTRACT_DIR}' has been deleted.")
else:
    print(f"Directory '{EXTRACT_DIR}' does not exist.")

Directory 'ocr' does not exist.


In [24]:
import os
import csv
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

In [ ]:
# Ensure output directory exists
output_dir = 'ocr'
os.makedirs(output_dir, exist_ok=True)

# Number of threads to use — adjust based on your CPU
MAX_WORKERS = 2

def process_image(image_path):
    try:
        # Skip if output already exists
        base_name = os.path.splitext(os.path.basename(image_path))[0]
        output_csv_path = os.path.join(output_dir, f"{base_name}.csv")
        if os.path.exists(output_csv_path):
            return (image_path, "Skipped (already exists)")

        ocr_result = easy_ocr(image_path)
        combined_text = " ".join(ocr_result.splitlines())

        with open(output_csv_path, mode='w', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow(['ocr_transf'])
            writer.writerow([combined_text])

        return (image_path, "Success")
    except Exception as e:
        return (image_path, f"Error: {e}")

# Submit tasks to the thread pool
with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    futures = [executor.submit(process_image, path) for path in png_paths]

    for future in tqdm(as_completed(futures), total=len(futures), desc="Parallel OCR with EasyOCR"):
        image_path, status = future.result()
        if "Error" in status:
            print(f"[!] Failed: {image_path} -> {status}")